<a href="https://colab.research.google.com/github/tseykoroman/Finance/blob/main/export_model_to_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file_path = '/content/drive/MyDrive/filename.txt'
with open(file_path, 'w') as f:
    f.write('This is the content of the file 22222.')


In [5]:
file_path = '/content/drive/MyDrive/filename.txt'
with open(file_path, 'r') as f:
    content = f.read()
print(content)

This is the content of the file 22222.


In [6]:
!pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.8/584.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 24.8 MB/s eta 0:00:00


In [20]:
import pandas as pd
import tensorflow as tf

import autokeras as ak

In [21]:
dataset = pd.read_csv('AirQualityUCI.csv', sep=";")
dataset = dataset[dataset.columns[:-2]]
dataset = dataset.dropna()
dataset = dataset.replace(",", ".", regex=True)

val_split = int(len(dataset) * 0.7)
data_train = dataset[:val_split]
validation_data = dataset[val_split:]

data_x = data_train[
    [
        "CO(GT)",
        "PT08.S1(CO)",
        "NMHC(GT)",
        "C6H6(GT)",
        "PT08.S2(NMHC)",
        "NOx(GT)",
        "PT08.S3(NOx)",
        "NO2(GT)",
        "PT08.S4(NO2)",
        "PT08.S5(O3)",
        "T",
        "RH",
    ]
].astype("float64")

data_x_val = validation_data[
    [
        "CO(GT)",
        "PT08.S1(CO)",
        "NMHC(GT)",
        "C6H6(GT)",
        "PT08.S2(NMHC)",
        "NOx(GT)",
        "PT08.S3(NOx)",
        "NO2(GT)",
        "PT08.S4(NO2)",
        "PT08.S5(O3)",
        "T",
        "RH",
    ]
].astype("float64")

# Data with train data and the unseen data from subsequent time steps.
data_x_test = dataset[
    [
        "CO(GT)",
        "PT08.S1(CO)",
        "NMHC(GT)",
        "C6H6(GT)",
        "PT08.S2(NMHC)",
        "NOx(GT)",
        "PT08.S3(NOx)",
        "NO2(GT)",
        "PT08.S4(NO2)",
        "PT08.S5(O3)",
        "T",
        "RH",
    ]
].astype("float64")

data_y = data_train["AH"].astype("float64")

data_y_val = validation_data["AH"].astype("float64")

print(data_x.shape)  # (6549, 12)
print(data_y.shape)  # (6549,)

(6549, 12)
(6549,)


In [40]:
predict_from = 1
predict_until = 1 #number of predicted items in future
lookback = 10
clf = ak.TimeseriesForecaster(
    lookback=lookback,
    predict_from=predict_from,
    predict_until=predict_until,
    max_trials=1,
    objective="val_loss",
)
# Train the TimeSeriesForecaster with train data
clf.fit(
    x=data_x,
    y=data_y,
    validation_data=(data_x_val, data_y_val),
    batch_size=32,
    epochs=5,
)
# Predict with the best model(includes original training data).
predictions = clf.predict(data_x_test)
print(predictions.shape)
# Evaluate the best model with testing data.
print(clf.evaluate(data_x_val, data_y_val))

Reloading Tuner from ./time_series_forecaster/tuner0.json
Epoch 1/5
205/205 [==============================] - 9s 25ms/step - loss: 901.1220 - mean_squared_error: 901.1220 - val_loss: 3099.3623 - val_mean_squared_error: 3099.3623
Epoch 2/5
205/205 [==============================] - 3s 12ms/step - loss: 897.4153 - mean_squared_error: 897.4153 - val_loss: 3091.1497 - val_mean_squared_error: 3091.1497
Epoch 3/5
205/205 [==============================] - 4s 21ms/step - loss: 894.2197 - mean_squared_error: 894.2197 - val_loss: 3086.6531 - val_mean_squared_error: 3086.6531
Epoch 4/5
205/205 [==============================] - 3s 13ms/step - loss: 892.2218 - mean_squared_error: 892.2218 - val_loss: 3075.1279 - val_mean_squared_error: 3075.1279
Epoch 5/5
293/293 [==============================] - 2s 7ms/step
(1, 1)
88/88 [==============================] - 1s 5ms/step - loss: 3069.0215 - mean_squared_error: 3069.0215
[3069.021484375, 3069.021484375]


In [42]:
# Export as a Keras Model.
model = clf.export_model()

print(type(model))  # <class 'tensorflow.python.keras.engine.training.Model'>


<class 'keras.src.engine.functional.Functional'>


In [43]:
try:
    model.save("/content/drive/MyDrive/model_autokeras", save_format="tf")
except Exception:
    model.save("/content/drive/MyDrive/model_autokeras.h5")


In [44]:
from tensorflow.keras.models import load_model

In [45]:
loaded_model = load_model("/content/drive/MyDrive/model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)

print(type(loaded_model))  # <class 'tensorflow.python.keras.engine.training.Model'>

<class 'keras.src.engine.functional.Functional'>
